In [1]:
import os
import requests
import os
import re
from bs4 import BeautifulSoup
import json
from tdmh import *

In [2]:
def get_categories(json_data):
    all_categories = []
    if 'text_has_categorie' in json_data['@relations']:
        categories = json_data['@relations']['text_has_categorie']
        for c in categories:
            if 'displayName' in c:
                all_categories.append( c['displayName'].lower() )
    return all_categories

def get_languages(json_data):
    all_lang = []
    if 'text_has_language' in json_data['@relations']:
        languages = json_data['@relations']['text_has_language']
        for l in languages:
            if 'displayName' in l:
                all_lang.append( l['displayName'].lower() )
    return all_lang

def get_owners(json_data):
    all_owners = []
    if 'textdrager_has_owner' in json_data['@relations']:
        owners = json_data['@relations']['textdrager_has_owner']
        for o in owners:
            if 'displayName' in o:
                all_owners.append( o['displayName'] )
    return all_owners

def get_copyists(json_data):
    all_copyists = []
    if 'textdrager_has_kopiist' in json_data['@relations']:
        copyists = json_data['@relations']['textdrager_has_kopiist']
        #print(copyists)
        for c in copyists:
            if c['relationTypeId'] != 'RELT000000000011':
                print( c['relationTypeId']  )
            if 'displayName' in c:
                all_copyists.append( c['displayName'] )
    return all_copyists


def get_authors(json_data):
    all_authors = []
    if 'tekstdragerHasTextOfAuthor' in json_data['@relations']:
        authors = json_data['@relations']['tekstdragerHasTextOfAuthor']
        for a in authors:
            if 'displayName' in a:
                all_authors.append( (a['id'],a['displayName']) )
    return all_authors

def get_binders(json_data):
    all_binders = []
    if 'textdrager_has_binder' in json_data['@relations']:
        binders = json_data['@relations']['textdrager_has_binder']
        for b in binders:
            if 'displayName' in b:
                all_binders.append( b['displayName']) 
    return all_binders



def get_text_carrier(json_data):
    carriers = []
    if 'text_part_of' in json_data['@relations']:
        for c in json_data['@relations']['text_part_of']:
            carriers.append(c['id'])
    return carriers


def intersection(list1, list2):
    intersection = [value for value in list1 if value in list2]
    return intersection

## Get all categories

In [3]:
files = os.listdir('BNM_texts')

all_categories = dict()

for file in files:
    if re.search( r'.txt$' , file ):
        #print(file)

        path = os.path.join('BNM_texts' , file)
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)
        
        categories = get_categories(json_data)
        for c in categories:
            all_categories[c] = all_categories.get(c,0)+1
            


In [4]:
for c in sortedByValue( all_categories , ascending = False ):
    print( f'{c} ({all_categories[c]})')

hagiografische teksten (2616)
gebeden (2519)
preken (2397)
getijdenboek (1213)
rijmspreuken (1208)
exempelen (1197)
berijmde tekst in versparen (1034)
exempelen over s. maria (727)
liederen (667)
kalenders voor bisdom utrecht (582)
dicta (488)
berijmde gebeden (483)
litanie (455)
sproken (441)
berijmde tekst in strofen (390)
communiegebeden (371)
gebedenboek (313)
berijmde tekst (301)
historiografische teksten (290)
medische teksten (286)
psalter (261)
kalenders (256)
medische recepten (248)
brevier (247)
mnl. vertalingen van de zeven boetpsalmen, in proza (240)
mnl. vertaling van de getijden van de eeuwige wijsheid (222)
mnl. vertalingen van bijbelteksten (206)
refreinen (187)
missaal (176)
prozatekst (176)
brieven (173)
juridische teksten (168)
getijden van s. maria (157)
computistische teksten en tabellen (147)
kronieken (133)
bijbel (113)
vigilie van de overledenen (107)
antifonarium (105)
glossarium (105)
getijden- en gebedenboek (104)
kloosterkronieken (99)
kloosterregels (93)
as

## Selected categories

In [9]:
i = 0 
for c in sortedByValue( all_categories , ascending = False ):
    if re.search( r'\bgetijden' , c ) or re.search( r'\bgebeden' , c ):
        i += 1
        print( f'{i}. {c} ({all_categories[c]})')

1. gebeden (2519)
2. getijdenboek (1213)
3. berijmde gebeden (483)
4. gebedenboek (313)
5. mnl. vertaling van de getijden van de eeuwige wijsheid (222)
6. getijden van s. maria (157)
7. getijden- en gebedenboek (104)
8. getijden van de heilige geest (91)
9. getijden (77)
10. getijden van de passie (65)
11. korte getijden van het h. kruis (52)
12. lange getijden van het h. kruis (46)
13. getijden van alle heiligen (33)
14. getijden van het heilig kruis (32)
15. gebedenkrans (25)
16. getijdenboek van geert grote (20)
17. getijden van de h. drievuldigheid (17)
18. getijden van de smarten van s. maria (12)
19. mnl. vertaling van de getijden van s. maria (8)
20. mnl. vertaling van lange getijden van het h. kruis (6)
21. mnl. vertaling van getijden van de heilige geest (6)
22. mnl. vertaling van korte getijden van het h. kruis (2)
23. vijf gebeden ter ere van de vijf wonden van jezus (1)


## Data model

## Tekst

* Auteur (niet in de dataset?)
* Titel
* Pagina in boek
* Signatuur
* Genormaliseerde titel
* Incipit


In [6]:
files = os.listdir('BNM_texts')

books = dict()

out = open( 'teksten.tsv' , 'w' , encoding = 'utf-8')

out.write( f'titel\tdatering\tpagina\tincipit\ttitel_norm\ttaal\n')

count = 0 
for file in files:
    if re.search( r'.txt$' , file ):
        #print(file)

        path = os.path.join('BNM_texts' , file)
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)
        
        categories = get_categories(json_data)
        for c in categories:
            if re.search( r'\bgetijden' , c ) or re.search( r'\bgebeden' , c ):
                count += 1
                text = json_data
                out.write( text['_id'] )
                out.write('\t')
                out.write( text['value'] )
                out.write('\t')
                if text['datering'] is not None:
                    out.write( text['datering'] )
                out.write('\t')
                out.write( text['@paginas'] )
                out.write('\t')
                if 'annotatie_incipit' in text and text['annotatie_incipit'] is not None:
                    out.write( text['annotatie_incipit'][0])
                out.write('\t')
                if 'text_has_norm_title' in json_data['@relations']:
                    norm_title = json_data['@relations']['text_has_norm_title']
                    out.write( norm_title[0]['displayName'] )
                out.write('\t')
                languages = get_languages(json_data)
                out.write( ';'.join(languages) )
                out.write('\t')
                carriers = get_text_carrier(json_data)
                out.write( ';'.join(carriers) )
                for book in carriers:
                    books[book]= books.get(book,0)+1
                out.write('\n')
                

out.close()


In [10]:
print( f'{count} texts selected.' )
print( f'These texts are in {len(books)} books.' )

5504 texts selected.
These texts are in 2731 books.


## Data model

## Boek

* Signatuur
* Titel
* Plaats
* Datering
* Taal
* Eigenaar
* Plaats boek productie (producent)
* Boekproductent: Kopiist, illustrator, ect. 



In [ ]:
dir = 'BNM_carriers'

out = open( 'tekstdragers.tsv' , 'w' , encoding = 'utf-8')

out.write('id\tsignatuur\taantal_teksten\tdatering\n')

files = os.listdir(dir)


for file in files:


    path = os.path.join( dir , file)
    
    try:
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)

        path = os.path.join('BNM_data' , file)
        
        nr_texts = 0 
        if 'contains_text' in json_data['@relations']:
            nr_texts = len( json_data['@relations']['contains_text'] )
        
        if json_data['_id'] in books:

            out.write( json_data['_id'] )
            out.write('\t')
            out.write( json_data['bewaarplaats_en_signatuur'] )
            out.write('\t')
            out.write(str(nr_texts))
            out.write('\t')
            if json_data['datering'] is not None:
                out.write( json_data['datering'] )
            out.write('\n')
    except:
        print(f'Problem in file {file}')

out.close()


## Eigenaar

In [11]:
dir = 'BNM_carriers'
out = open( 'eigenaars.tsv' , 'w' , encoding = 'utf-8')

files = os.listdir(dir)


for file in files:
    if re.search( r'txt$' , file ):

        path = os.path.join( dir , file)
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)

        path = os.path.join('BNM_data' , file)

        if json_data['_id'] in books:

            owners = get_owners(json_data)
            for o in owners:
                out.write( f"{json_data['_id']}\t{json_data['bewaarplaats_en_signatuur']}\t{o}\n" )
            
            
out.close()

## Copyist

In [12]:
dir = 'BNM_carriers'


out = open( 'kopiist.tsv' , 'w' , encoding = 'utf-8')
out.write('id\tsignatuur\tkopiist\n')

files = os.listdir(dir)


for file in files:
    if re.search( r'txt$' , file ):
        path = os.path.join( dir , file)
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)

        path = os.path.join('BNM_data' , file)

        if json_data['_id'] in books:

            copyists = get_copyists(json_data)
            for c in copyists:
                out.write( f"{json_data['_id']}\t{json_data['bewaarplaats_en_signatuur']}\t{c}\n" )
            
out.close()

## Auteurs

In [13]:
dir = 'BNM_carriers'

out = open( 'auteurs.tsv' , 'w' , encoding = 'utf-8')

out.write('id\tsignatuur\taut_id\tauteur\n')

files = os.listdir(dir)


for file in files:
    if re.search( r'txt$' , file ):

        path = os.path.join( dir , file)
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)

        path = os.path.join('BNM_data' , file)

        if json_data['_id'] in books:

            authors = get_authors(json_data)
            for a in authors:
                out.write( f"{json_data['_id']}\t{json_data['bewaarplaats_en_signatuur']}\t{a[0]}\t{a[1]}\n" )
            
            
out.close()

## Data for gephi

In [ ]:
import pandas as pd



df = pd.read_csv('auteurs.tsv', sep = '\t' )

df = df.rename(columns={'id':"Source",
                        'signatuur': "Source_label",
                        'aut_id':"Target",
                        'auteur': "Target_label"})

df1 = df.drop(['Target','Target_label'], axis=1)
df1 = df1.rename(columns={'Source':"id",'Source_label':"Label"})
df1['Type'] = 'Book'

df2 = df.drop(['Source','Source_label'], axis=1)
df2 = df2.rename(columns={'Target':"id",'Target_label':"Label"})
df2['Type'] = 'Author'

df_nodes = pd.concat( [df1, df2] )
df_nodes.head()
df_nodes.to_csv('nodes.csv')

df_edges = df.copy()
df_edges = df_edges.drop(['Source_label','Target_label'], axis=1)
df_edges.to_csv('edges.csv')

## Binders

In [14]:
dir = 'BNM_carriers'

out = open( 'binders.tsv' , 'w' , encoding = 'utf-8')

files = os.listdir(dir)


for file in files:
    if re.search( r'txt$' , file ):

        path = os.path.join( dir , file)
        json_str = open( path , encoding = 'utf-8')
        json_data = json.load(json_str)

        path = os.path.join('BNM_data' , file)

        if json_data['_id'] in books:

            binders = get_binders(json_data)
            for b in binders:
                out.write( f"{json_data['_id']}\t{json_data['bewaarplaats_en_signatuur']}\t{b}\n" )
            
            
out.close()